In [104]:
import csv
import pandas as pd
import datetime as dt
from datetime import datetime
from itertools import islice
import numpy as np
from openpyxl import load_workbook
import collections
import openpyxl as op
import math
import xlsxwriter

In [105]:
exportFile = 'Model Running.xlsx'
writer = pd.ExcelWriter(exportFile, engine = 'xlsxwriter')

In [106]:
def datevalue(date1):
    temp = dt.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

def valuedate(fh):
    fh = int(fh)
    return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + fh - 2)

In [107]:
df_input = pd.read_csv("Input Running.csv")
inputList = df_input["Values"].tolist()
df_input

,Var Names,Values
0,File Name,GABI_BGN Query3 8-8-18.XLSX
1,Predict Thru Year,2019
2,Predict Thru Month,8
3,Predict Thru Date,8


In [108]:
predict_thru = dt.datetime(int (inputList[1]), int (inputList[2]), int (inputList[3]))
predict_thru = datevalue(predict_thru)

In [109]:
df_query3 = pd.read_excel(inputList[0])
df_query3

,Constituent ID,Name,First Gift Date,Gift Date,Preferred State,Gift Average Amount_1,Gift Amount,Gift Type
0,1753793,"Klee, Joseph C.",36094,36094,Ohio,778.0555,5.0,Cash
1,1753793,"Klee, Joseph C.",36094,37329,Ohio,778.0555,200.0,Cash
2,1753793,"Klee, Joseph C.",36094,37432,Ohio,778.0555,100.0,Cash
3,1753793,"Klee, Joseph C.",36094,37558,Ohio,778.0555,50.0,Cash
4,1753793,"Klee, Joseph C.",36094,38197,Ohio,778.0555,200.0,Cash
...,...,...,...,...,...,...,...,...
80091,1934508,"Rodriguez, Jaclyn",43310,43310,Texas,50.0000,50.0,Cash
80092,1934509,"McAluney, Matthew",43312,43312,Georgia,1000.0000,1000.0,Cash
80093,1934511,"PAES, ROYCE",43312,43312,Georgia,25.0000,25.0,Cash
80094,1934510,"Elliott, Susan",43313,43313,Georgia,42.0000,42.0,Cash


In [110]:
dfraw1 = df_query3.copy(deep=True)
removePledge0 = []
for index, rows in dfraw1.iterrows():
    if(dfraw1.iloc[index, 6] == 0 or dfraw1.iloc[index, 7] == ("Pledge")):
        removePledge0.append(True)
    else:
        removePledge0.append("")
dfraw1['remove $0 & pledges'] = (removePledge0)
dfraw1

,Constituent ID,Name,First Gift Date,Gift Date,Preferred State,Gift Average Amount_1,Gift Amount,Gift Type,remove $0 & pledges
0,1753793,"Klee, Joseph C.",36094,36094,Ohio,778.0555,5.0,Cash,
1,1753793,"Klee, Joseph C.",36094,37329,Ohio,778.0555,200.0,Cash,
2,1753793,"Klee, Joseph C.",36094,37432,Ohio,778.0555,100.0,Cash,
3,1753793,"Klee, Joseph C.",36094,37558,Ohio,778.0555,50.0,Cash,
4,1753793,"Klee, Joseph C.",36094,38197,Ohio,778.0555,200.0,Cash,
...,...,...,...,...,...,...,...,...,...
80091,1934508,"Rodriguez, Jaclyn",43310,43310,Texas,50.0000,50.0,Cash,
80092,1934509,"McAluney, Matthew",43312,43312,Georgia,1000.0000,1000.0,Cash,
80093,1934511,"PAES, ROYCE",43312,43312,Georgia,25.0000,25.0,Cash,
80094,1934510,"Elliott, Susan",43313,43313,Georgia,42.0000,42.0,Cash,


In [111]:
dfdp1 = dfraw1.copy(deep=True)
pledgeor0 = dfdp1[dfdp1['remove $0 & pledges'] == True].index
dfdp1.drop(pledgeor0, inplace=True)
dfdp1 = dfdp1.reset_index(drop=True)
dfdp1

,Constituent ID,Name,First Gift Date,Gift Date,Preferred State,Gift Average Amount_1,Gift Amount,Gift Type,remove $0 & pledges
0,1753793,"Klee, Joseph C.",36094,36094,Ohio,778.0555,5.0,Cash,
1,1753793,"Klee, Joseph C.",36094,37329,Ohio,778.0555,200.0,Cash,
2,1753793,"Klee, Joseph C.",36094,37432,Ohio,778.0555,100.0,Cash,
3,1753793,"Klee, Joseph C.",36094,37558,Ohio,778.0555,50.0,Cash,
4,1753793,"Klee, Joseph C.",36094,38197,Ohio,778.0555,200.0,Cash,
...,...,...,...,...,...,...,...,...,...
79935,1934508,"Rodriguez, Jaclyn",43310,43310,Texas,50.0000,50.0,Cash,
79936,1934509,"McAluney, Matthew",43312,43312,Georgia,1000.0000,1000.0,Cash,
79937,1934511,"PAES, ROYCE",43312,43312,Georgia,25.0000,25.0,Cash,
79938,1934510,"Elliott, Susan",43313,43313,Georgia,42.0000,42.0,Cash,


In [112]:
dfraw2 = dfdp1.copy(deep=True)
dfraw2 = dfraw2.drop(columns = ["Gift Amount", "Gift Type", "remove $0 & pledges"])
dfraw2

,Constituent ID,Name,First Gift Date,Gift Date,Preferred State,Gift Average Amount_1
0,1753793,"Klee, Joseph C.",36094,36094,Ohio,778.0555
1,1753793,"Klee, Joseph C.",36094,37329,Ohio,778.0555
2,1753793,"Klee, Joseph C.",36094,37432,Ohio,778.0555
3,1753793,"Klee, Joseph C.",36094,37558,Ohio,778.0555
4,1753793,"Klee, Joseph C.",36094,38197,Ohio,778.0555
...,...,...,...,...,...,...
79935,1934508,"Rodriguez, Jaclyn",43310,43310,Texas,50.0000
79936,1934509,"McAluney, Matthew",43312,43312,Georgia,1000.0000
79937,1934511,"PAES, ROYCE",43312,43312,Georgia,25.0000
79938,1934510,"Elliott, Susan",43313,43313,Georgia,42.0000


In [113]:
adGiftList = [1]
for index, row in islice(dfraw2.iterrows(), 1, None):
    if(dfraw2.iloc[index-1, 0] != dfraw2.iloc[index, 0]):
        adGiftList.append(1)
    else:
        if(dfraw2.iloc[index-1, 0] == dfraw2.iloc[index, 0] and dfraw2.iloc[index-1, 3] != dfraw2.iloc[index, 3]):
            adGiftList.append(adGiftList[index-1] + 1)
        else:
            adGiftList.append(adGiftList[index-1])

dfraw2['adj. num gift'] = (adGiftList)

In [114]:
len(dfraw2)

79940

In [115]:
keepList = []
for index, row in islice(dfraw2.iterrows(), 0, None):
    try:
        if (index+1 >= len(dfraw2)):
            keepList.append("")
        elif (dfraw2.iloc[index-1,0] == dfraw2.iloc[index,0] and dfraw2.iloc[index,0] != dfraw2.iloc[index+1,0]):
            keepList.append("")
        else:
            if (dfraw2.iloc[index-1,0] != dfraw2.iloc[index,0] and dfraw2.iloc[index,0] != dfraw2.iloc[index+1,0]):
                keepList.append("")
            else:
                keepList.append(False) 
    except(IndexError):
        print(index)
        
dfraw2['KEEP'] = (keepList)
dfraw2

,Constituent ID,Name,First Gift Date,Gift Date,Preferred State,Gift Average Amount_1,adj. num gift,KEEP
0,1753793,"Klee, Joseph C.",36094,36094,Ohio,778.0555,1,False
1,1753793,"Klee, Joseph C.",36094,37329,Ohio,778.0555,2,False
2,1753793,"Klee, Joseph C.",36094,37432,Ohio,778.0555,3,False
3,1753793,"Klee, Joseph C.",36094,37558,Ohio,778.0555,4,False
4,1753793,"Klee, Joseph C.",36094,38197,Ohio,778.0555,5,False
...,...,...,...,...,...,...,...,...
79935,1934508,"Rodriguez, Jaclyn",43310,43310,Texas,50.0000,1,
79936,1934509,"McAluney, Matthew",43312,43312,Georgia,1000.0000,1,
79937,1934511,"PAES, ROYCE",43312,43312,Georgia,25.0000,1,
79938,1934510,"Elliott, Susan",43313,43313,Georgia,42.0000,1,


In [116]:
dfdp2 = dfraw2.copy(deep=True)
falseKeep = dfdp2[dfdp2['KEEP'] == False].index
dfdp2.drop(falseKeep, inplace=True)
dfdp2 = dfdp2.reset_index(drop=True)
dfdp2

,Constituent ID,Name,First Gift Date,Gift Date,Preferred State,Gift Average Amount_1,adj. num gift,KEEP
0,1753793,"Klee, Joseph C.",36094,42200,Ohio,778.0555,18,
1,1447060,"Burke, Trevor",36222,42475,California,31.0294,34,
2,1624463,"Dooley, Eugene A.",36229,43236,New York,5.2631,19,
3,1198365,"Wiktorowski, Richard M.",36230,42927,Michigan,11.5714,14,
4,1492526,"White, Timothy J.",36230,41893,Arizona,18.8750,40,
...,...,...,...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",43310,43310,Texas,50.0000,1,
4316,1934509,"McAluney, Matthew",43312,43312,Georgia,1000.0000,1,
4317,1934511,"PAES, ROYCE",43312,43312,Georgia,25.0000,1,
4318,1934510,"Elliott, Susan",43313,43313,Georgia,42.0000,1,


In [117]:
grgFile = "GRG Running.xlsx"
df_GRG = pd.read_excel(grgFile, header=None, nrows = 5, usecols = [0,1])
r = df_GRG.iloc[0,1]
alpha = df_GRG.iloc[1,1]
a = df_GRG.iloc[2,1]
b = df_GRG.iloc[3,1]
print(r, alpha, a, b)

0.14764008054195857 0.3515710413869862 0.3278680524073309 1.3670175730326175


In [118]:
dfmd = dfdp2[["Constituent ID", "Name"]]
dfmd["x (#donations)"] = dfdp2["adj. num gift"]-1
dfmd['t_x (last gift)'] = (dfdp2['Gift Date'] - dfdp2['First Gift Date'])/365

<ipython-input-118-ea6a9f9cc4ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd["x (#donations)"] = dfdp2["adj. num gift"]-1
<ipython-input-118-ea6a9f9cc4ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd['t_x (last gift)'] = (dfdp2['Gift Date'] - dfdp2['First Gift Date'])/365


In [119]:
today = pd.to_datetime("today")
today = round(datevalue(today),0)
tempDate = (today)+1
print(tempDate)
dfmd['T (total time span)'] = (tempDate - dfdp2['First Gift Date'])/365
dfmd

44422.0


<ipython-input-119-ddb7301598a8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmd['T (total time span)'] = (tempDate - dfdp2['First Gift Date'])/365


,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span)
0,1753793,"Klee, Joseph C.",17,16.728767,22.816438
1,1447060,"Burke, Trevor",33,17.131507,22.465753
2,1624463,"Dooley, Eugene A.",18,19.197260,22.446575
3,1198365,"Wiktorowski, Richard M.",13,18.347945,22.443836
4,1492526,"White, Timothy J.",39,15.515068,22.443836
...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",0,0.000000,3.046575
4316,1934509,"McAluney, Matthew",0,0.000000,3.041096
4317,1934511,"PAES, ROYCE",0,0.000000,3.041096
4318,1934510,"Elliott, Susan",0,0.000000,3.038356


In [120]:
df_allCondExp = dfmd.copy(deep=True)
print(predict_thru, today)
constT = (predict_thru-today)/365
print(constT)
df_allCondExp["T (total time span)"] = df_allCondExp["T (total time span)"]-3
df_allCondExp['t'] = constT
df_allCondExp["E(Y(t)|X=x,t_x,T)"] = ""
df_allCondExp["2F1"] = 0
df_allCondExp["a"] = df_allCondExp.iloc[:,2]+r
df_allCondExp["b"] = df_allCondExp.iloc[:,2] + b
df_allCondExp["c"] = df_allCondExp.iloc[:,2] + b + a - 1
df_allCondExp["z"] = constT/(alpha + df_allCondExp.iloc[:,4] + constT)
df_allCondExp['0'] = 1
df_allCondExp

43685.0 44421.0
-2.0164383561643837


,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,c,z,0
0,1753793,"Klee, Joseph C.",17,16.728767,19.816438,-2.016438,,0,17.14764,18.367018,17.694886,-0.111089,1
1,1447060,"Burke, Trevor",33,17.131507,19.465753,-2.016438,,0,33.14764,34.367018,33.694886,-0.113277,1
2,1624463,"Dooley, Eugene A.",18,19.197260,19.446575,-2.016438,,0,18.14764,19.367018,18.694886,-0.113400,1
3,1198365,"Wiktorowski, Richard M.",13,18.347945,19.443836,-2.016438,,0,13.14764,14.367018,13.694886,-0.113417,1
4,1492526,"White, Timothy J.",39,15.515068,19.443836,-2.016438,,0,39.14764,40.367018,39.694886,-0.113417,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",0,0.000000,0.046575,-2.016438,,0,0.14764,1.367018,0.694886,1.246029,1
4316,1934509,"McAluney, Matthew",0,0.000000,0.041096,-2.016438,,0,0.14764,1.367018,0.694886,1.241824,1
4317,1934511,"PAES, ROYCE",0,0.000000,0.041096,-2.016438,,0,0.14764,1.367018,0.694886,1.241824,1
4318,1934510,"Elliott, Susan",0,0.000000,0.038356,-2.016438,,0,0.14764,1.367018,0.694886,1.239732,1


In [121]:
aceCount = 1
aceArr = []
while (aceCount <= 200):
    df_allCondExp[str(aceCount)]=df_allCondExp.iloc[:,aceCount+11]*(df_allCondExp.iloc[:,8]+aceCount-1)*(df_allCondExp.iloc[:,9]+aceCount-1)/((df_allCondExp.iloc[:,10]+aceCount-1)*aceCount)*df_allCondExp.iloc[:,11]
    aceCount = aceCount + 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1753793,"Klee, Joseph C.",17,16.728767,19.816438,-2.016438,,0,17.14764,18.367018,...,-1.214922e-158,1.467864e-159,-1.772728e-160,2.140025e-161,-2.582367e-162,3.114878e-163,-3.755691e-164,4.526539e-165,-5.453454e-166,6.567618e-167
1,1447060,"Burke, Trevor",33,17.131507,19.465753,-2.016438,,0,33.14764,34.367018,...,-5.377344e-142,7.132491e-143,-9.453358e-144,1.252003e-144,-1.656924e-145,2.191191e-146,-2.895621e-147,3.823754e-148,-5.045772e-149,6.653619e-150
2,1624463,"Dooley, Eugene A.",18,19.197260,19.446575,-2.016438,,0,18.14764,19.367018,...,-7.271156e-156,9.010657e-157,-1.116139e-157,1.381946e-158,-1.710316e-159,2.115807e-160,-2.616328e-161,3.233897e-162,-3.995583e-163,4.934641e-164
3,1198365,"Wiktorowski, Richard M.",13,18.347945,19.443836,-2.016438,,0,13.14764,14.367018,...,-1.907480e-161,2.307838e-162,-2.791321e-163,3.375007e-164,-4.079450e-165,4.929376e-166,-5.954521e-167,7.190644e-168,-8.680726e-169,1.047642e-169
4,1492526,"White, Timothy J.",39,15.515068,19.443836,-2.016438,,0,39.14764,40.367018,...,-4.123426e-137,5.622189e-138,-7.659034e-139,1.042480e-139,-1.417717e-140,1.926388e-141,-2.615371e-142,3.547820e-143,-4.808748e-144,6.512503e-145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",0,0.000000,0.046575,-2.016438,,0,0.14764,1.367018,...,1.589293e+17,1.978426e+17,2.462849e+17,3.065900e+17,3.816631e+17,4.751214e+17,5.914677e+17,7.363080e+17,9.166215e+17,1.141097e+18
4316,1934509,"McAluney, Matthew",0,0.000000,0.041096,-2.016438,,0,0.14764,1.367018,...,8.333225e+16,1.033858e+17,1.282658e+17,1.591340e+17,1.974319e+17,2.449479e+17,3.039010e+17,3.770445e+17,4.677945e+17,5.803896e+17
4317,1934511,"PAES, ROYCE",0,0.000000,0.041096,-2.016438,,0,0.14764,1.367018,...,8.333225e+16,1.033858e+17,1.282658e+17,1.591340e+17,1.974319e+17,2.449479e+17,3.039010e+17,3.770445e+17,4.677945e+17,5.803896e+17
4318,1934510,"Elliott, Susan",0,0.000000,0.038356,-2.016438,,0,0.14764,1.367018,...,6.039101e+16,7.479766e+16,9.264157e+16,1.147429e+17,1.421176e+17,1.760241e+17,2.180211e+17,2.700392e+17,3.344700e+17,4.142758e+17


In [122]:
aceCount1 = 0
while (aceCount1 <= 200):
    df_allCondExp["2F1"] = df_allCondExp["2F1"]+df_allCondExp.iloc[:,aceCount1+12]
    aceCount1 = aceCount1 + 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1753793,"Klee, Joseph C.",17,16.728767,19.816438,-2.016438,,1.533793e-01,17.14764,18.367018,...,-1.214922e-158,1.467864e-159,-1.772728e-160,2.140025e-161,-2.582367e-162,3.114878e-163,-3.755691e-164,4.526539e-165,-5.453454e-166,6.567618e-167
1,1447060,"Burke, Trevor",33,17.131507,19.465753,-2.016438,,2.657174e-02,33.14764,34.367018,...,-5.377344e-142,7.132491e-143,-9.453358e-144,1.252003e-144,-1.656924e-145,2.191191e-146,-2.895621e-147,3.823754e-148,-5.045772e-149,6.653619e-150
2,1624463,"Dooley, Eugene A.",18,19.197260,19.446575,-2.016438,,1.327407e-01,18.14764,19.367018,...,-7.271156e-156,9.010657e-157,-1.116139e-157,1.381946e-158,-1.710316e-159,2.115807e-160,-2.616328e-161,3.233897e-162,-3.995583e-163,4.934641e-164
3,1198365,"Wiktorowski, Richard M.",13,18.347945,19.443836,-2.016438,,2.272570e-01,13.14764,14.367018,...,-1.907480e-161,2.307838e-162,-2.791321e-163,3.375007e-164,-4.079450e-165,4.929376e-166,-5.954521e-167,7.190644e-168,-8.680726e-169,1.047642e-169
4,1492526,"White, Timothy J.",39,15.515068,19.443836,-2.016438,,1.388513e-02,39.14764,40.367018,...,-4.123426e-137,5.622189e-138,-7.659034e-139,1.042480e-139,-1.417717e-140,1.926388e-141,-2.615371e-142,3.547820e-143,-4.808748e-144,6.512503e-145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",0,0.000000,0.046575,-2.016438,,5.801111e+18,0.14764,1.367018,...,1.589293e+17,1.978426e+17,2.462849e+17,3.065900e+17,3.816631e+17,4.751214e+17,5.914677e+17,7.363080e+17,9.166215e+17,1.141097e+18
4316,1934509,"McAluney, Matthew",0,0.000000,0.041096,-2.016438,,2.991962e+18,0.14764,1.367018,...,8.333225e+16,1.033858e+17,1.282658e+17,1.591340e+17,1.974319e+17,2.449479e+17,3.039010e+17,3.770445e+17,4.677945e+17,5.803896e+17
4317,1934511,"PAES, ROYCE",0,0.000000,0.041096,-2.016438,,2.991962e+18,0.14764,1.367018,...,8.333225e+16,1.033858e+17,1.282658e+17,1.591340e+17,1.974319e+17,2.449479e+17,3.039010e+17,3.770445e+17,4.677945e+17,5.803896e+17
4318,1934510,"Elliott, Susan",0,0.000000,0.038356,-2.016438,,2.150709e+18,0.14764,1.367018,...,6.039101e+16,7.479766e+16,9.264157e+16,1.147429e+17,1.421176e+17,1.760241e+17,2.180211e+17,2.700392e+17,3.344700e+17,4.142758e+17


In [123]:
longNameArr = []
#((a+b+df_allCondExp.iloc[:,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[:,4])/(alpha+df_allCondExp.iloc[:,4]+df_allCondExp.iloc[:,5])),(r+df_allCondExp.iloc[:,2]))*df_allCondExp.iloc[:,7])/(1+(df_allCondExp.iloc[:,2]>0)*a/(b+df_allCondExp.iloc[:,2]-1)*pow(((alpha+df_allCondExp.iloc[:,4])/(alpha+df_allCondExp.iloc[:,3])),(r+df_allCondExp.iloc[:,2]))))
for index, row in islice(df_allCondExp.iterrows(), 0, None):
    longNameArr.append((a+b+df_allCondExp.iloc[index,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,4]+df_allCondExp.iloc[index,5])),(r+df_allCondExp.iloc[index,2]))*df_allCondExp.iloc[index,7])/(1+(df_allCondExp.iloc[index,2]>0)*a/(b+df_allCondExp.iloc[index,2]-1)*pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,3])),(r+df_allCondExp.iloc[index,2]))))
df_allCondExp["E(Y(t)|X=x,t_x,T)"] = longNameArr
df_allCondExp

<ipython-input-123-9d21aaa3111e>:4: RuntimeWarning: invalid value encountered in double_scalars
  longNameArr.append((a+b+df_allCondExp.iloc[index,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,4]+df_allCondExp.iloc[index,5])),(r+df_allCondExp.iloc[index,2]))*df_allCondExp.iloc[index,7])/(1+(df_allCondExp.iloc[index,2]>0)*a/(b+df_allCondExp.iloc[index,2]-1)*pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,3])),(r+df_allCondExp.iloc[index,2]))))


,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1753793,"Klee, Joseph C.",17,16.728767,19.816438,-2.016438,-1.314283,1.533793e-01,17.14764,18.367018,...,-1.214922e-158,1.467864e-159,-1.772728e-160,2.140025e-161,-2.582367e-162,3.114878e-163,-3.755691e-164,4.526539e-165,-5.453454e-166,6.567618e-167
1,1447060,"Burke, Trevor",33,17.131507,19.465753,-2.016438,-2.110077,2.657174e-02,33.14764,34.367018,...,-5.377344e-142,7.132491e-143,-9.453358e-144,1.252003e-144,-1.656924e-145,2.191191e-146,-2.895621e-147,3.823754e-148,-5.045772e-149,6.653619e-150
2,1624463,"Dooley, Eugene A.",18,19.197260,19.446575,-2.016438,-1.838433,1.327407e-01,18.14764,19.367018,...,-7.271156e-156,9.010657e-157,-1.116139e-157,1.381946e-158,-1.710316e-159,2.115807e-160,-2.616328e-161,3.233897e-162,-3.995583e-163,4.934641e-164
3,1198365,"Wiktorowski, Richard M.",13,18.347945,19.443836,-2.016438,-1.294652,2.272570e-01,13.14764,14.367018,...,-1.907480e-161,2.307838e-162,-2.791321e-163,3.375007e-164,-4.079450e-165,4.929376e-166,-5.954521e-167,7.190644e-168,-8.680726e-169,1.047642e-169
4,1492526,"White, Timothy J.",39,15.515068,19.443836,-2.016438,-0.082673,1.388513e-02,39.14764,40.367018,...,-4.123426e-137,5.622189e-138,-7.659034e-139,1.042480e-139,-1.417717e-140,1.926388e-141,-2.615371e-142,3.547820e-143,-4.808748e-144,6.512503e-145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",0,0.000000,0.046575,-2.016438,NaN,5.801111e+18,0.14764,1.367018,...,1.589293e+17,1.978426e+17,2.462849e+17,3.065900e+17,3.816631e+17,4.751214e+17,5.914677e+17,7.363080e+17,9.166215e+17,1.141097e+18
4316,1934509,"McAluney, Matthew",0,0.000000,0.041096,-2.016438,NaN,2.991962e+18,0.14764,1.367018,...,8.333225e+16,1.033858e+17,1.282658e+17,1.591340e+17,1.974319e+17,2.449479e+17,3.039010e+17,3.770445e+17,4.677945e+17,5.803896e+17
4317,1934511,"PAES, ROYCE",0,0.000000,0.041096,-2.016438,NaN,2.991962e+18,0.14764,1.367018,...,8.333225e+16,1.033858e+17,1.282658e+17,1.591340e+17,1.974319e+17,2.449479e+17,3.039010e+17,3.770445e+17,4.677945e+17,5.803896e+17
4318,1934510,"Elliott, Susan",0,0.000000,0.038356,-2.016438,NaN,2.150709e+18,0.14764,1.367018,...,6.039101e+16,7.479766e+16,9.264157e+16,1.147429e+17,1.421176e+17,1.760241e+17,2.180211e+17,2.700392e+17,3.344700e+17,4.142758e+17


In [124]:
df_pAlive = df_allCondExp.iloc[:,0:5].copy(deep=True)
paliveinfo = []
for index, row in islice(df_allCondExp.iterrows(), 0, None):
    paliveinfo.append(1/(1+(df_pAlive.iloc[index,2]>0)*(a/(b+df_pAlive.iloc[index,2]-1))*(pow(((alpha+df_pAlive.iloc[index,4])/(alpha+df_pAlive.iloc[index,3])),(r+df_pAlive.iloc[index,2])))))
df_pAlive["P(Alive) Info"] = paliveinfo
df_pAlive

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info
0,1753793,"Klee, Joseph C.",17,16.728767,19.816438,0.754044
1,1447060,"Burke, Trevor",33,17.131507,19.465753,0.615040
2,1624463,"Dooley, Eugene A.",18,19.197260,19.446575,0.978027
3,1198365,"Wiktorowski, Richard M.",13,18.347945,19.443836,0.950694
4,1492526,"White, Timothy J.",39,15.515068,19.443836,0.020380
...,...,...,...,...,...,...
4315,1934508,"Rodriguez, Jaclyn",0,0.000000,0.046575,1.000000
4316,1934509,"McAluney, Matthew",0,0.000000,0.041096,1.000000
4317,1934511,"PAES, ROYCE",0,0.000000,0.041096,1.000000
4318,1934510,"Elliott, Susan",0,0.000000,0.038356,1.000000


In [125]:
dfraw2.to_excel(writer, 'Raw data2', index = False)
dfraw1.to_excel(writer, 'Raw data1', index = False)
dfraw1.to_excel(writer, 'Data prep1', index = False)
dfdp2.to_excel(writer, 'Data prep2', index = False)
dfmd.to_excel(writer, 'Model data', index = False)
df_allCondExp.to_excel(writer, "All Cond. Exp.", index=False)
df_pAlive.to_excel(writer, "P(alive)", index=False)
writer.save()